<a href="https://colab.research.google.com/github/ReisK1ng/ProjetoBanco/blob/main/ProjetoBanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Rafael Leonardo dos Reis Freitas

RGM: 37743449

Curso: Ciência da Computação

Turma: D1

Link do repositório Git


In [ ]:
# -*- coding: utf-8 -*-

# === Importações de Módulos ===
import sqlite3  # Biblioteca para interagir com o banco de dados SQLite
import os       # Módulo para interagir com o sistema operacional (não utilizado diretamente, mas bom para futuras expansões)
import re       # Módulo de expressões regulares, usado para validar o CPF
from abc import ABC, abstractmethod  # Ferramentas para criar classes abstratas
from collections.abc import Iterable, Sized # Interfaces para coleções iteráveis e com tamanho definido

# === Exceções Personalizadas ===
# Define exceções específicas para o domínio do problema, tornando o tratamento de erros mais claro.

class ErroDeConta(Exception):
    """Exceção base para todos os erros relacionados a contas no sistema bancário."""
    pass

class SaldoInsuficienteError(ErroDeConta):
    """Exceção lançada quando uma operação de saque excede o saldo disponível."""
    pass

class OperacaoInvalidaError(ErroDeConta):
    """Exceção para operações que não são permitidas, como valores negativos ou dados inválidos."""
    pass

class ContaNaoEncontradaError(ErroDeConta):
    """Exceção lançada quando uma busca por número de conta não retorna resultados."""
    pass

# === Classe Cliente ===
class Cliente:
    """Representa um cliente do banco, contendo nome e CPF."""
    def __init__(self, nome, cpf):
        """
        Inicializa um novo cliente.
        Valida o formato do CPF para garantir que tenha 11 dígitos.
        """
        # Validação do CPF usando expressão regular para aceitar apenas 11 dígitos numéricos
        if not re.fullmatch(r"\d{11}", cpf):
            raise OperacaoInvalidaError("CPF inválido: deve conter 11 dígitos")
        self.__nome = nome
        self.__cpf = cpf

    def __str__(self):
        """Retorna uma representação em string do cliente."""
        return f'{self.__nome} ({self.__cpf})'

    # Propriedades para acessar os atributos privados de forma segura (somente leitura)
    @property
    def nome(self):
        return self.__nome

    @property
    def cpf(self):
        return self.__cpf

# === Classe Abstrata Conta ===
# Define a estrutura base para todos os tipos de conta. Não pode ser instanciada diretamente.
class Conta(ABC):
    _total_contas = 0  # Atributo de classe para contar o total de contas criadas

    def __init__(self, numero, cliente, saldo=0.0):
        """
        Inicializador da conta. Garante que o cliente é uma instância válida.
        """
        if not isinstance(cliente, Cliente):
            raise OperacaoInvalidaError("Cliente inválido")
        self._numero = numero
        self._cliente = cliente
        self.__saldo = max(saldo, 0.0)  # Garante que o saldo inicial não seja negativo
        Conta._total_contas += 1

    # Propriedades para acesso controlado aos atributos
    @property
    def numero(self):
        return self._numero

    @property
    def cliente(self):
        return self._cliente

    @property
    def saldo(self):
        return self.__saldo

    @saldo.setter
    def saldo(self, valor):
        """Setter para o saldo que impede a atribuição de um valor negativo."""
        if valor < 0:
            raise OperacaoInvalidaError("Saldo não pode ser negativo")
        self.__saldo = valor

    @classmethod
    def total_contas(cls):
        """Retorna o número total de contas criadas."""
        return cls._total_contas

    def depositar(self, valor):
        """Adiciona um valor ao saldo da conta."""
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de depósito deve ser positivo")
        self.__saldo += valor

    def sacar(self, valor):
        """
        Retira um valor do saldo da conta após validação.
        Este método é sobrescrito nas classes filhas.
        """
        self._validar_saque(valor)
        self.saldo -= valor

    def _validar_saque(self, valor):
        """
        Método auxiliar para validar a operação de saque.
        Verifica se o valor é positivo e se há saldo suficiente.
        """
        if valor <= 0:
            raise OperacaoInvalidaError("Valor de saque deve ser positivo")
        if valor > self.saldo:
            raise SaldoInsuficienteError("Saldo insuficiente")

    def transferir(self, destino, valor):
        """Transfere um valor desta conta para uma conta de destino."""
        if not isinstance(destino, Conta):
            raise OperacaoInvalidaError("Conta de destino inválida")
        # Reutiliza a lógica de saque e depósito
        self.sacar(valor)
        destino.depositar(valor)

    def extrato(self):
        """Retorna uma string com as informações da conta e o saldo atual."""
        return f'Conta {self._numero} - {self._cliente.nome} - Saldo: R${self.__saldo:.2f}'

    @abstractmethod
    def atualiza(self, taxa):
        """
        Método abstrato para atualização de saldo com base em uma taxa.
        Deve ser implementado pelas classes filhas.
        """
        pass

    # Esta implementação de `atualiza` na classe base permite reutilização de código
    def atualiza(self, taxa):
        """Implementação base da atualização de saldo."""
        if taxa < 0:
            raise OperacaoInvalidaError("Taxa não pode ser negativa")
        self.__saldo += self.__saldo * taxa

# === Conta Corrente ===
class ContaCorrente(Conta):
    """Implementação de uma conta corrente."""
    def sacar(self, valor):
        """Sobrescreve o método sacar para a lógica específica da conta corrente."""
        self._validar_saque(valor)
        self.saldo -= valor

    def depositar(self, valor):
        """Sobrescreve o depósito para cobrar uma taxa de R$ 0,10 por operação."""
        super().depositar(valor - 0.10)

    def atualiza(self, taxa):
        """Aplica a taxa de atualização em dobro para contas correntes."""
        super().atualiza(taxa * 2)

# === Conta Poupança ===
class ContaPoupanca(Conta):
    """Implementação de uma conta poupança."""
    def sacar(self, valor):
        """Sobrescreve o método sacar para a lógica específica da conta poupança."""
        self._validar_saque(valor)
        self.saldo -= valor

    def atualiza(self, taxa):
        """Aplica a taxa de atualização em triplo para contas poupança."""
        super().atualiza(taxa * 3)

# === Banco ===
# Classe que gerencia todas as contas e a persistência de dados com o SQLite.
# Herda de Iterable e Sized para permitir iteração (ex: for conta in banco) e uso de len().
class Banco(Iterable, Sized):
    def __init__(self, db_name="banco.db"):
        """Inicializa o banco, definindo o nome do arquivo de banco de dados e configurando as tabelas."""
        self.__db_name = db_name
        self.__setup_database()

    def __setup_database(self):
        """Configura o banco de dados, criando as tabelas 'clientes' e 'contas' se não existirem."""
        with sqlite3.connect(self.__db_name) as conn:
            cursor = conn.cursor()

            # Lógica simples de "migração": verifica se a tabela antiga existe sem a coluna `cpf_cliente`.
            # Se sim, apaga as tabelas antigas para criar a nova estrutura.
            # Isso é útil para desenvolvimento, mas em produção seria necessária uma migração mais robusta.
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contas'")
            tabela_antiga = cursor.fetchone()
            if tabela_antiga:
                cursor.execute("PRAGMA table_info(contas)")
                colunas = [info[1] for info in cursor.fetchall()]
                if 'cpf_cliente' not in colunas:
                    cursor.execute("DROP TABLE IF EXISTS clientes")
                    cursor.execute("DROP TABLE IF EXISTS contas")
                    conn.commit()

            # Cria a tabela de clientes se ela não existir
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS clientes (
                    cpf TEXT PRIMARY KEY,
                    nome TEXT NOT NULL
                )
            """)
            # Cria a tabela de contas se ela não existir, com chave estrangeira para clientes
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS contas (
                    numero INTEGER PRIMARY KEY,
                    cpf_cliente TEXT NOT NULL,
                    tipo TEXT NOT NULL,
                    saldo REAL NOT NULL,
                    FOREIGN KEY (cpf_cliente) REFERENCES clientes(cpf)
                )
            """)
            conn.commit()

    def __conectar(self):
        """Método privado para estabelecer uma conexão com o banco de dados."""
        return sqlite3.connect(self.__db_name)

    def adicionar_conta(self, conta):
        """Adiciona uma nova conta ao banco de dados."""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            # Verifica se o cliente já existe antes de inseri-lo para evitar erros de chave primária.
            cursor.execute("SELECT 1 FROM clientes WHERE cpf = ?", (conta.cliente.cpf,))
            if not cursor.fetchone():
                cursor.execute(
                    "INSERT INTO clientes (cpf, nome) VALUES (?, ?)",
                    (conta.cliente.cpf, conta.cliente.nome)
                )

            # Insere a conta na tabela 'contas'
            cursor.execute(
                "INSERT INTO contas (numero, cpf_cliente, tipo, saldo) VALUES (?, ?, ?, ?)",
                (conta.numero, conta.cliente.cpf, conta.__class__.__name__, conta.saldo)
            )
            conn.commit()

    def buscar_conta(self, numero):
        """Busca uma conta pelo número e a recria como um objeto Python."""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            # Une as tabelas 'contas' e 'clientes' para obter todos os dados necessários
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                WHERE c.numero = ?
            """, (numero,))
            dados = cursor.fetchone()

            if not dados:
                raise ContaNaoEncontradaError(f"Conta {numero} não encontrada")

            # Recria os objetos Cliente e Conta com base nos dados do banco
            cliente = Cliente(nome=dados[1], cpf=dados[2])
            tipo_conta = dados[3]
            if tipo_conta == "ContaCorrente":
                return ContaCorrente(numero=dados[0], cliente=cliente, saldo=dados[4])
            elif tipo_conta == "ContaPoupanca":
                return ContaPoupanca(numero=dados[0], cliente=cliente, saldo=dados[4])
            else:
                raise OperacaoInvalidaError(f"Tipo de conta inválido no banco de dados: {tipo_conta}")

    def listar_contas(self):
        """Retorna uma lista de tuplas com os dados de todas as contas no banco."""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT c.numero, cl.nome, cl.cpf, c.tipo, c.saldo
                FROM contas c
                JOIN clientes cl ON c.cpf_cliente = cl.cpf
                ORDER BY c.numero
            """)
            return cursor.fetchall()

    def atualizar_saldo(self, numero, saldo):
        """Atualiza o saldo de uma conta específica no banco de dados."""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute(
                "UPDATE contas SET saldo = ? WHERE numero = ?",
                (saldo, numero)
            )
            conn.commit()

    def remover_conta(self, numero):
        """Remove uma conta do banco de dados pelo seu número."""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            # O cliente não é removido, mesmo que não tenha mais contas.
            # Uma melhoria seria implementar um garbage collector para clientes órfãos.
            cursor.execute("DELETE FROM contas WHERE numero = ?", (numero,))
            conn.commit()

    def __iter__(self):
        """Permite a iteração sobre as contas do banco. Ex: for conta in banco: ..."""
        contas_data = self.listar_contas()
        for conta_data in contas_data:
            # `yield` transforma o método em um gerador, que busca e cria cada objeto de conta sob demanda.
            yield self.buscar_conta(conta_data[0])

    def __len__(self):
        """Permite o uso da função len() no objeto Banco. Ex: len(banco)"""
        with self.__conectar() as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM contas")
            # fetchone()[0] pega o primeiro valor da primeira linha do resultado
            return cursor.fetchone()[0]

# === Menu CLI (Interface de Linha de Comando) ===
def menu():
    """Função principal que executa a interface de linha de comando para o usuário."""
    banco = Banco()  # Instancia o banco

    while True:
        # Exibe o menu de opções
        print("\n==== MENU BANCÁRIO ====")
        print("1. Criar conta")
        print("2. Depositar")
        print("3. Sacar")
        print("4. Transferir")
        print("5. Listar contas")
        print("6. Remover conta")
        print("7. Sair")

        try:
            opcao = input("Escolha uma opção: ").strip()

            # O `match-case` (disponível no Python 3.10+) direciona para a lógica correta
            match opcao:
                case "1":
                    # Coleta os dados para criar uma nova conta
                    nome = input("Nome do cliente: ").strip()
                    cpf = input("CPF (11 dígitos): ").strip()
                    tipo = input("Tipo (corrente/poupança): ").strip().lower()
                    numero = int(input("Número da conta: "))
                    saldo_inicial = float(input("Saldo inicial: R$ "))

                    # Cria os objetos Cliente e Conta
                    cliente = Cliente(nome, cpf)
                    if tipo == "corrente":
                        conta = ContaCorrente(numero, cliente, saldo_inicial)
                    elif tipo == "poupança":
                        conta = ContaPoupanca(numero, cliente, saldo_inicial)
                    else:
                        print("Tipo de conta inválido!")
                        continue # Volta para o início do loop

                    banco.adicionar_conta(conta)
                    print(f"Conta {numero} criada com sucesso!")

                case "2":
                    # Lógica para depósito
                    numero = int(input("Número da conta: "))
                    valor = float(input("Valor a depositar: R$ "))
                    conta = banco.buscar_conta(numero)  # Busca a conta no banco
                    conta.depositar(valor)              # Realiza a operação no objeto
                    banco.atualizar_saldo(numero, conta.saldo) # Persiste a alteração no banco
                    print(f"Depósito de R$ {valor:.2f} realizado com sucesso!")

                case "3":
                    # Lógica para saque
                    numero = int(input("Número da conta: "))
                    valor = float(input("Valor a sacar: R$ "))
                    conta = banco.buscar_conta(numero)
                    conta.sacar(valor)
                    banco.atualizar_saldo(numero, conta.saldo)
                    print(f"Saque de R$ {valor:.2f} realizado com sucesso!")

                case "4":
                    # Lógica para transferência
                    origem = int(input("Conta de origem: "))
                    destino = int(input("Conta de destino: "))
                    valor = float(input("Valor a transferir: R$ "))

                    # Busca ambas as contas
                    conta_origem = banco.buscar_conta(origem)
                    conta_destino = banco.buscar_conta(destino)

                    # Realiza a transferência
                    conta_origem.transferir(conta_destino, valor)

                    # Atualiza o saldo de ambas as contas no banco de dados
                    banco.atualizar_saldo(origem, conta_origem.saldo)
                    banco.atualizar_saldo(destino, conta_destino.saldo)
                    print(f"Transferência de R$ {valor:.2f} realizada com sucesso!")

                case "5":
                    # Lista todas as contas existentes
                    print("\n=== LISTA DE CONTAS ===")
                    contas = banco.listar_contas()
                    if not contas:
                        print("Nenhuma conta cadastrada.")
                    else:
                        for conta_data in contas:
                            print(f"Conta {conta_data[0]} - {conta_data[1]} (CPF: {conta_data[2]})")
                            print(f"  Tipo: {conta_data[3]} - Saldo: R$ {conta_data[4]:.2f}")
                    print("======================")

                case "6":
                    # Remove uma conta
                    numero = int(input("Número da conta a remover: "))
                    banco.remover_conta(numero)
                    print(f"Conta {numero} removida com sucesso!")

                case "7":
                    # Encerra o programa
                    print("Encerrando sistema bancário...")
                    break # Sai do loop while

                case _:
                    # Caso o usuário digite uma opção inválida
                    print("Opção inválida! Tente novamente.")

        # Bloco de tratamento de exceções
        except ValueError:
            print("Erro: Valor inválido fornecido! Certifique-se de digitar números quando solicitado.")
        except SaldoInsuficienteError as e:
            print(f"Erro de Operação: {e}")
        except OperacaoInvalidaError as e:
            print(f"Erro de Operação: {e}")
        except ContaNaoEncontradaError as e:
            print(f"Erro de Busca: {e}")
        except Exception as e:
            # Captura qualquer outro erro inesperado para evitar que o programa quebre
            print(f"Erro inesperado: {e}")

# Ponto de entrada do script
# A verificação `if __name__ == "__main__":` garante que a função `menu()`
# só será executada quando o script for rodado diretamente.
if __name__ == "__main__":
    menu()